In [15]:
# #@markdown ### **Installing pip packages**
# #@markdown - Diffusion Model: [PyTorch](https://pytorch.org) & [HuggingFace diffusers](https://huggingface.co/docs/diffusers/index)
# #@markdown - Dataset Loading: [Zarr](https://zarr.readthedocs.io/en/stable/) & numcodecs
# #@markdown - Push-T Env: gym, pygame, pymunk & shapely
# !python --version
# !pip3 install zarr scikit-video torch==1.13.1 torchvision==0.14.1 diffusers==0.18.2 \
# scikit-image==0.19.3 scikit-video==1.1.11 zarr==2.12.0 numcodecs==0.10.2 \
# pygame==2.1.2 pymunk==6.2.1 gym==0.26.2 shapely==1.8.4 \
# &> /dev/null # mute output
# # Reinstall pymunk explicitly to ensure it's correctly installed
# !pip install pymunk==6.2.1

In [16]:
#@markdown ### **Imports**
# diffusion policy import
from typing import Tuple, Sequence, Dict, Union, Optional, Callable
import numpy as np
import math
import torch
import torch.nn as nn
import torchvision
import collections
import zarr
# from diffusers.schedulers.scheduling_ddpm import DDPMScheduler
# from diffusers.training_utils import EMAModel
# from diffusers.optimization import get_scheduler
from tqdm.auto import tqdm

# env import
import gym
from gym import spaces
import pygame
import pymunk
import pymunk.pygame_util
from pymunk.space_debug_draw_options import SpaceDebugColor
from pymunk.vec2d import Vec2d
import shapely.geometry as sg
import cv2
import skimage.transform as st
from skvideo.io import vwrite
from IPython.display import Video
# import gdown
import os

In [17]:
#@markdown ### **Environment**
#@markdown Defines a PyMunk-based Push-T environment `PushTEnv`.
#@markdown And it's subclass `PushTImageEnv`.
#@markdown
#@markdown **Goal**: push the gray T-block into the green area.
#@markdown
#@markdown Adapted from [Implicit Behavior Cloning](https://implicitbc.github.io/)


positive_y_is_up: bool = False
"""Make increasing values of y point upwards.

When True::

    y
    ^
    |      . (3, 3)
    |
    |   . (2, 2)
    |
    +------ > x

When False::

    +------ > x
    |
    |   . (2, 2)
    |
    |      . (3, 3)
    v
    y

"""

def to_pygame(p: Tuple[float, float], surface: pygame.Surface) -> Tuple[int, int]:
    """Convenience method to convert pymunk coordinates to pygame surface
    local coordinates.

    Note that in case positive_y_is_up is False, this function wont actually do
    anything except converting the point to integers.
    """
    if positive_y_is_up:
        return round(p[0]), surface.get_height() - round(p[1])
    else:
        return round(p[0]), round(p[1])


def light_color(color: SpaceDebugColor):
    color = np.minimum(1.2 * np.float32([color.r, color.g, color.b, color.a]), np.float32([255]))
    color = SpaceDebugColor(r=color[0], g=color[1], b=color[2], a=color[3])
    return color

class DrawOptions(pymunk.SpaceDebugDrawOptions):
    def __init__(self, surface: pygame.Surface) -> None:
        """Draw a pymunk.Space on a pygame.Surface object.

        Typical usage::

        >>> import pymunk
        >>> surface = pygame.Surface((10,10))
        >>> space = pymunk.Space()
        >>> options = pymunk.pygame_util.DrawOptions(surface)
        >>> space.debug_draw(options)

        You can control the color of a shape by setting shape.color to the color
        you want it drawn in::

        >>> c = pymunk.Circle(None, 10)
        >>> c.color = pygame.Color("pink")

        See pygame_util.demo.py for a full example

        Since pygame uses a coordiante system where y points down (in contrast
        to many other cases), you either have to make the physics simulation
        with Pymunk also behave in that way, or flip everything when you draw.

        The easiest is probably to just make the simulation behave the same
        way as Pygame does. In that way all coordinates used are in the same
        orientation and easy to reason about::

        >>> space = pymunk.Space()
        >>> space.gravity = (0, -1000)
        >>> body = pymunk.Body()
        >>> body.position = (0, 0) # will be positioned in the top left corner
        >>> space.debug_draw(options)

        To flip the drawing its possible to set the module property
        :py:data:`positive_y_is_up` to True. Then the pygame drawing will flip
        the simulation upside down before drawing::

        >>> positive_y_is_up = True
        >>> body = pymunk.Body()
        >>> body.position = (0, 0)
        >>> # Body will be position in bottom left corner

        :Parameters:
                surface : pygame.Surface
                    Surface that the objects will be drawn on
        """
        self.surface = surface
        super(DrawOptions, self).__init__()

    def draw_circle(
        self,
        pos: Vec2d,
        angle: float,
        radius: float,
        outline_color: SpaceDebugColor,
        fill_color: SpaceDebugColor,
    ) -> None:
        p = to_pygame(pos, self.surface)

        pygame.draw.circle(self.surface, fill_color.as_int(), p, round(radius), 0)
        pygame.draw.circle(self.surface, light_color(fill_color).as_int(), p, round(radius-4), 0)

        circle_edge = pos + Vec2d(radius, 0).rotated(angle)
        p2 = to_pygame(circle_edge, self.surface)
        line_r = 2 if radius > 20 else 1
        # pygame.draw.lines(self.surface, outline_color.as_int(), False, [p, p2], line_r)

    def draw_segment(self, a: Vec2d, b: Vec2d, color: SpaceDebugColor) -> None:
        p1 = to_pygame(a, self.surface)
        p2 = to_pygame(b, self.surface)

        pygame.draw.aalines(self.surface, color.as_int(), False, [p1, p2])

    def draw_fat_segment(
        self,
        a: Tuple[float, float],
        b: Tuple[float, float],
        radius: float,
        outline_color: SpaceDebugColor,
        fill_color: SpaceDebugColor,
    ) -> None:
        p1 = to_pygame(a, self.surface)
        p2 = to_pygame(b, self.surface)

        r = round(max(1, radius * 2))
        pygame.draw.lines(self.surface, fill_color.as_int(), False, [p1, p2], r)
        if r > 2:
            orthog = [abs(p2[1] - p1[1]), abs(p2[0] - p1[0])]
            if orthog[0] == 0 and orthog[1] == 0:
                return
            scale = radius / (orthog[0] * orthog[0] + orthog[1] * orthog[1]) ** 0.5
            orthog[0] = round(orthog[0] * scale)
            orthog[1] = round(orthog[1] * scale)
            points = [
                (p1[0] - orthog[0], p1[1] - orthog[1]),
                (p1[0] + orthog[0], p1[1] + orthog[1]),
                (p2[0] + orthog[0], p2[1] + orthog[1]),
                (p2[0] - orthog[0], p2[1] - orthog[1]),
            ]
            pygame.draw.polygon(self.surface, fill_color.as_int(), points)
            pygame.draw.circle(
                self.surface,
                fill_color.as_int(),
                (round(p1[0]), round(p1[1])),
                round(radius),
            )
            pygame.draw.circle(
                self.surface,
                fill_color.as_int(),
                (round(p2[0]), round(p2[1])),
                round(radius),
            )

    def draw_polygon(
        self,
        verts: Sequence[Tuple[float, float]],
        radius: float,
        outline_color: SpaceDebugColor,
        fill_color: SpaceDebugColor,
    ) -> None:
        ps = [to_pygame(v, self.surface) for v in verts]
        ps += [ps[0]]

        radius = 2
        pygame.draw.polygon(self.surface, light_color(fill_color).as_int(), ps)

        if radius > 0:
            for i in range(len(verts)):
                a = verts[i]
                b = verts[(i + 1) % len(verts)]
                self.draw_fat_segment(a, b, radius, fill_color, fill_color)

    def draw_dot(
        self, size: float, pos: Tuple[float, float], color: SpaceDebugColor
    ) -> None:
        p = to_pygame(pos, self.surface)
        pygame.draw.circle(self.surface, color.as_int(), p, round(size), 0)


def pymunk_to_shapely(body, shapes):
    geoms = list()
    for shape in shapes:
        if isinstance(shape, pymunk.shapes.Poly):
            verts = [body.local_to_world(v) for v in shape.get_vertices()]
            verts += [verts[0]]
            geoms.append(sg.Polygon(verts))
        else:
            raise RuntimeError(f'Unsupported shape type {type(shape)}')
    geom = sg.MultiPolygon(geoms)
    return geom

# env
class PushTEnv(gym.Env):
    metadata = {"render.modes": ["human", "rgb_array"], "video.frames_per_second": 10}
    reward_range = (0., 1.)

    def __init__(self,
            legacy=False,
            block_cog=None, damping=None,
            render_action=True,
            render_size=96,
            reset_to_state=None
        ):
        self._seed = None
        self.seed()
        self.window_size = ws = 512  # The size of the PyGame window
        self.render_size = render_size
        self.sim_hz = 100
        # Local controller params.
        self.k_p, self.k_v = 100, 20    # PD control.z
        self.control_hz = self.metadata['video.frames_per_second']
        # legcay set_state for data compatiblity
        self.legacy = legacy

        # agent_pos, block_pos, block_angle
        self.observation_space = spaces.Box(
            low=np.array([0,0,0,0,0], dtype=np.float64),
            high=np.array([ws,ws,ws,ws,np.pi*2], dtype=np.float64),
            shape=(5,),
            dtype=np.float64
        )

        # positional goal for agent
        self.action_space = spaces.Box(
            low=np.array([0,0], dtype=np.float64),
            high=np.array([ws,ws], dtype=np.float64),
            shape=(2,),
            dtype=np.float64
        )

        self.block_cog = block_cog
        self.damping = damping
        self.render_action = render_action

        """
        If human-rendering is used, `self.window` will be a reference
        to the window that we draw to. `self.clock` will be a clock that is used
        to ensure that the environment is rendered at the correct framerate in
        human-mode. They will remain `None` until human-mode is used for the
        first time.
        """
        self.window = None
        self.clock = None
        self.screen = None

        self.space = None
        self.teleop = None
        self.render_buffer = None
        self.latest_action = None
        self.reset_to_state = reset_to_state

    def reset(self):
        seed = self._seed
        self._setup()
        if self.block_cog is not None:
            self.block.center_of_gravity = self.block_cog
        if self.damping is not None:
            self.space.damping = self.damping

        # use legacy RandomState for compatiblity
        state = self.reset_to_state
        if state is None:
            rs = np.random.RandomState(seed=seed)
            state = np.array([
                rs.randint(50, 450), rs.randint(50, 450),
                rs.randint(100, 400), rs.randint(100, 400),
                rs.randn() * 2 * np.pi - np.pi
                ])
        self._set_state(state)

        obs = self._get_obs()
        info = self._get_info()
        return obs, info

    def step(self, action):
        dt = 1.0 / self.sim_hz
        self.n_contact_points = 0
        n_steps = self.sim_hz // self.control_hz
        if action is not None:
            self.latest_action = action
            for i in range(n_steps):
                # Step PD control.
                # self.agent.velocity = self.k_p * (act - self.agent.position)    # P control works too.
                acceleration = self.k_p * (action - self.agent.position) + self.k_v * (Vec2d(0, 0) - self.agent.velocity)
                self.agent.velocity += acceleration * dt

                # Step physics.
                self.space.step(dt)

        # compute reward
        goal_body = self._get_goal_pose_body(self.goal_pose)
        goal_geom = pymunk_to_shapely(goal_body, self.block.shapes)
        block_geom = pymunk_to_shapely(self.block, self.block.shapes)

        intersection_area = goal_geom.intersection(block_geom).area
        goal_area = goal_geom.area
        coverage = intersection_area / goal_area
        reward = np.clip(coverage / self.success_threshold, 0, 1)
        done = coverage > self.success_threshold
        terminated = done
        truncated = done

        observation = self._get_obs()
        info = self._get_info()

        return observation, reward, terminated, truncated, info

    def render(self, mode):
        return self._render_frame(mode)

    def teleop_agent(self):
        TeleopAgent = collections.namedtuple('TeleopAgent', ['act'])
        def act(obs):
            act = None
            mouse_position = pymunk.pygame_util.from_pygame(Vec2d(*pygame.mouse.get_pos()), self.screen)
            if self.teleop or (mouse_position - self.agent.position).length < 30:
                self.teleop = True
                act = mouse_position
            return act
        return TeleopAgent(act)

    def _get_obs(self):
        obs = np.array(
            tuple(self.agent.position) \
            + tuple(self.block.position) \
            + (self.block.angle % (2 * np.pi),))
        return obs

    def _get_goal_pose_body(self, pose):
        mass = 1
        inertia = pymunk.moment_for_box(mass, (50, 100))
        body = pymunk.Body(mass, inertia)
        # preserving the legacy assignment order for compatibility
        # the order here dosn't matter somehow, maybe because CoM is aligned with body origin
        body.position = pose[:2].tolist()
        body.angle = pose[2]
        return body

    def _get_info(self):
        n_steps = self.sim_hz // self.control_hz
        n_contact_points_per_step = int(np.ceil(self.n_contact_points / n_steps))
        info = {
            'pos_agent': np.array(self.agent.position),
            'vel_agent': np.array(self.agent.velocity),
            'block_pose': np.array(list(self.block.position) + [self.block.angle]),
            'goal_pose': self.goal_pose,
            'n_contacts': n_contact_points_per_step}
        return info

    def _render_frame(self, mode):

        if self.window is None and mode == "human":
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode((self.window_size, self.window_size))
        if self.clock is None and mode == "human":
            self.clock = pygame.time.Clock()

        canvas = pygame.Surface((self.window_size, self.window_size))
        canvas.fill((255, 255, 255))
        self.screen = canvas

        draw_options = DrawOptions(canvas)

        # Draw goal pose.
        goal_body = self._get_goal_pose_body(self.goal_pose)
        for shape in self.block.shapes:
            goal_points = [pymunk.pygame_util.to_pygame(goal_body.local_to_world(v), draw_options.surface) for v in shape.get_vertices()]
            goal_points += [goal_points[0]]
            pygame.draw.polygon(canvas, self.goal_color, goal_points)

        # Draw agent and block.
        self.space.debug_draw(draw_options)

        if mode == "human":
            # The following line copies our drawings from `canvas` to the visible window
            self.window.blit(canvas, canvas.get_rect())
            pygame.event.pump()
            pygame.display.update()

            # the clock is aleady ticked during in step for "human"


        img = np.transpose(
                np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
            )
        img = cv2.resize(img, (self.render_size, self.render_size))
        if self.render_action:
            if self.render_action and (self.latest_action is not None):
                action = np.array(self.latest_action)
                coord = (action / 512 * 96).astype(np.int32)
                marker_size = int(8/96*self.render_size)
                thickness = int(1/96*self.render_size)
                cv2.drawMarker(img, coord,
                    color=(255,0,0), markerType=cv2.MARKER_CROSS,
                    markerSize=marker_size, thickness=thickness)
        return img


    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()

    def seed(self, seed=None):
        if seed is None:
            seed = np.random.randint(0,25536)
        self._seed = seed
        self.np_random = np.random.default_rng(seed)

    def _handle_collision(self, arbiter, space, data):
        self.n_contact_points += len(arbiter.contact_point_set.points)

    def _set_state(self, state):
        if isinstance(state, np.ndarray):
            state = state.tolist()
        pos_agent = state[:2]
        pos_block = state[2:4]
        rot_block = state[4]
        self.agent.position = pos_agent
        # setting angle rotates with respect to center of mass
        # therefore will modify the geometric position
        # if not the same as CoM
        # therefore should be modified first.
        if self.legacy:
            # for compatiblity with legacy data
            self.block.position = pos_block
            self.block.angle = rot_block
        else:
            self.block.angle = rot_block
            self.block.position = pos_block

        # Run physics to take effect
        self.space.step(1.0 / self.sim_hz)

    def _set_state_local(self, state_local):
        agent_pos_local = state_local[:2]
        block_pose_local = state_local[2:]
        tf_img_obj = st.AffineTransform(
            translation=self.goal_pose[:2],
            rotation=self.goal_pose[2])
        tf_obj_new = st.AffineTransform(
            translation=block_pose_local[:2],
            rotation=block_pose_local[2]
        )
        tf_img_new = st.AffineTransform(
            matrix=tf_img_obj.params @ tf_obj_new.params
        )
        agent_pos_new = tf_img_new(agent_pos_local)
        new_state = np.array(
            list(agent_pos_new[0]) + list(tf_img_new.translation) \
                + [tf_img_new.rotation])
        self._set_state(new_state)
        return new_state

    def _setup(self):
        self.space = pymunk.Space()
        self.space.gravity = 0, 0
        self.space.damping = 0
        self.teleop = False
        self.render_buffer = list()

        # Add walls.
        walls = [
            self._add_segment((5, 506), (5, 5), 2),
            self._add_segment((5, 5), (506, 5), 2),
            self._add_segment((506, 5), (506, 506), 2),
            self._add_segment((5, 506), (506, 506), 2)
        ]
        self.space.add(*walls)

        # Add agent, block, and goal zone.
        self.agent = self.add_circle((256, 400), 15)
        self.block = self.add_tee((256, 300), 0)
        self.goal_color = pygame.Color('LightGreen')
        self.goal_pose = np.array([256,256,np.pi/4])  # x, y, theta (in radians)

        # Add collision handeling
        self.collision_handeler = self.space.add_collision_handler(0, 0)
        self.collision_handeler.post_solve = self._handle_collision
        self.n_contact_points = 0

        self.max_score = 50 * 100
        self.success_threshold = 0.95    # 95% coverage.

    def _add_segment(self, a, b, radius):
        shape = pymunk.Segment(self.space.static_body, a, b, radius)
        shape.color = pygame.Color('LightGray')    # https://htmlcolorcodes.com/color-names
        return shape

    def add_circle(self, position, radius):
        body = pymunk.Body(body_type=pymunk.Body.KINEMATIC)
        body.position = position
        body.friction = 1
        shape = pymunk.Circle(body, radius)
        shape.color = pygame.Color('RoyalBlue')
        self.space.add(body, shape)
        return body

    def add_box(self, position, height, width):
        mass = 1
        inertia = pymunk.moment_for_box(mass, (height, width))
        body = pymunk.Body(mass, inertia)
        body.position = position
        shape = pymunk.Poly.create_box(body, (height, width))
        shape.color = pygame.Color('LightSlateGray')
        self.space.add(body, shape)
        return body

    def add_tee(self, position, angle, scale=30, color='LightSlateGray', mask=pymunk.ShapeFilter.ALL_MASKS()):
        mass = 1
        length = 4
        vertices1 = [(-length*scale/2, scale),
                                 ( length*scale/2, scale),
                                 ( length*scale/2, 0),
                                 (-length*scale/2, 0)]
        inertia1 = pymunk.moment_for_poly(mass, vertices=vertices1)
        vertices2 = [(-scale/2, scale),
                                 (-scale/2, length*scale),
                                 ( scale/2, length*scale),
                                 ( scale/2, scale)]
        inertia2 = pymunk.moment_for_poly(mass, vertices=vertices1)
        body = pymunk.Body(mass, inertia1 + inertia2)
        shape1 = pymunk.Poly(body, vertices1)
        shape2 = pymunk.Poly(body, vertices2)
        shape1.color = pygame.Color(color)
        shape2.color = pygame.Color(color)
        shape1.filter = pymunk.ShapeFilter(mask=mask)
        shape2.filter = pymunk.ShapeFilter(mask=mask)
        body.center_of_gravity = (shape1.center_of_gravity + shape2.center_of_gravity) / 2
        body.position = position
        body.angle = angle
        body.friction = 1
        self.space.add(body, shape1, shape2)
        return body


class PushTImageEnv(PushTEnv):
    metadata = {"render.modes": ["rgb_array"], "video.frames_per_second": 10}

    def __init__(self,
            legacy=False,
            block_cog=None,
            damping=None,
            render_size=96):
        super().__init__(
            legacy=legacy,
            block_cog=block_cog,
            damping=damping,
            render_size=render_size,
            render_action=False)
        ws = self.window_size
        self.observation_space = spaces.Dict({
            'image': spaces.Box(
                low=0,
                high=1,
                shape=(3,render_size,render_size),
                dtype=np.float32
            ),
            'agent_pos': spaces.Box(
                low=0,
                high=ws,
                shape=(2,),
                dtype=np.float32
            )
        })
        self.render_cache = None

    def _get_obs(self):
        img = super()._render_frame(mode='rgb_array')

        agent_pos = np.array(self.agent.position)
        img_obs = np.moveaxis(img.astype(np.float32) / 255, -1, 0)
        obs = {
            'image': img_obs,
            'agent_pos': agent_pos
        }

        # draw action
        if self.latest_action is not None:
            action = np.array(self.latest_action)
            coord = (action / 512 * 96).astype(np.int32)
            marker_size = int(8/96*self.render_size)
            thickness = int(1/96*self.render_size)
            cv2.drawMarker(img, coord,
                color=(255,0,0), markerType=cv2.MARKER_CROSS,
                markerSize=marker_size, thickness=thickness)
        self.render_cache = img

        return obs

    def render(self, mode):
        assert mode == 'rgb_array'

        if self.render_cache is None:
            self._get_obs()

        return self.render_cache



In [18]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import os # For creating directories if needed

def plot_and_save_grayscale(output_2d_tensor, filename="grayscale_output.png", title="Autoencoder Output (Grayscale)"):
    plt.ioff()
    """
    Plots a 1xHxW tensor as a grayscale image and saves it.
    Scales values from -1.0 to 1.0 to 0.0 to 1.0 for plotting.

    Args:
        output_2d_tensor (torch.Tensor): The output tensor of shape (1, H, W) or (H, W).
                                         Values are expected to be between -1.0 and 1.0.
        filename (str): The name of the file to save the plot.
        title (str): The title of the plot.
    """
    if output_2d_tensor.dim() == 3 and output_2d_tensor.shape[0] == 1:
        # Remove the channel dimension if it's 1
        plot_data = output_2d_tensor.squeeze(0).cpu().numpy()
    elif output_2d_tensor.dim() == 2:
        plot_data = output_2d_tensor.cpu().numpy()
    else:
        raise ValueError("output_2d_tensor must be of shape (1, H, W) or (H, W)")

    # Scale values from [-1, 1] to [0, 1] for typical image display
    plot_data_scaled = (plot_data + 1.0) / 2.0

    plt.figure(figsize=(6, 6))
    plt.imshow(plot_data_scaled, cmap='gray', vmin=0, vmax=1)
    plt.title(title)
    plt.colorbar(label="Scaled Value (0.0 to 1.0)")
    plt.axis('off') # Hide axes for cleaner image
    plt.tight_layout()
    plt.savefig(filename)
    plt.close() # Close the plot to free memory

    # print(f"Grayscale plot saved to {filename}")

def plot_and_save_heatmap(output_2d_tensor, filename="heatmap_output.png", title="Autoencoder Output (Heatmap)"):
    plt.ioff()
    """
    Plots a 1xHxW tensor as a heatmap and saves it.
    Values are expected to be between -1.0 and 1.0.

    Args:
        output_2d_tensor (torch.Tensor): The output tensor of shape (1, H, W) or (H, W).
                                         Values are expected to be between -1.0 and 1.0.
        filename (str): The name of the file to save the plot.
        title (str): The title of the plot.
    """
    if output_2d_tensor.dim() == 3 and output_2d_tensor.shape[0] == 1:
        # Remove the channel dimension if it's 1
        plot_data = output_2d_tensor.squeeze(0).cpu().numpy()
    elif output_2d_tensor.dim() == 2:
        plot_data = output_2d_tensor.cpu().numpy()
    else:
        raise ValueError("output_2d_tensor must be of shape (1, H, W) or (H, W)")

    plt.figure(figsize=(7, 6)) # Slightly wider for colorbar
    plt.imshow(plot_data, cmap='viridis', vmin=-1.0, vmax=1.0) # 'viridis' or 'plasma' are good perceptual colormaps
    plt.title(title)
    plt.colorbar(label="Value (-1.0 to 1.0)")
    plt.axis('off') # Hide axes for cleaner image
    plt.tight_layout()
    plt.savefig(filename)
    plt.close() # Close the plot to free memory

    # print(f"Heatmap plot saved to {filename}")
def viz(dummy_output_2d):
    output_dir = "model_outputs"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Plot and save as grayscale
    plot_and_save_grayscale(dummy_output_2d, filename=os.path.join(output_dir, "output_grayscale.png"))

    # Plot and save as heatmap
    plot_and_save_heatmap(dummy_output_2d, filename=os.path.join(output_dir, "output_heatmap.png"))


In [19]:
#@markdown ### **Env Demo**
#@markdown Standard Gym Env (0.21.0 API)

# 0. create env object
env = PushTImageEnv()

# 1. seed env for initial state.
# Seed 0-200 are used for the demonstration dataset.
env.seed(1000)

# 2. must reset before use
obs, info = env.reset()

# 3. 2D positional action space [0,512]
action = env.action_space.sample()

# 4. Standard gym step method
obs, reward, terminated, truncated, info = env.step(action)

# prints and explains each dimension of the observation and action vectors
with np.printoptions(precision=4, suppress=True, threshold=5):
    print("obs['image'].shape:", obs['image'].shape, "float32, [0,1]")
    print("obs['agent_pos'].shape:", obs['agent_pos'].shape, "float32, [0,512]")
    print("action.shape: ", action.shape, "float32, [0,512]")

obs['image'].shape: (3, 96, 96) float32, [0,1]
obs['agent_pos'].shape: (2,) float32, [0,512]
action.shape:  (2,) float32, [0,512]


In [ ]:
def plot_agent_positions(naction,path, batch_idx=0, max_trajectories=8, figsize=(15, 10)):
    plt.ioff()
    if isinstance(naction, torch.Tensor):
        naction_np = naction.detach().cpu().numpy()
    else:
        naction_np = naction
    if isinstance(path, torch.Tensor):
        path = path.detach().cpu().numpy()
    else:
        path = path
    

    B, T, coord_dim = naction_np.shape
    # print(f"Agent position tensor shape: {naction_np.shape}")
    # print(f"Batch size: {B}, Time steps: {T}, Coordinates: {coord_dim}")
    
    # Create subplots
    fig = plt.figure(figsize=figsize)
    
    plt.subplot(2, 3, 1)
    traj = naction_np[batch_idx]  # Shape: (T, 2)
    x_coords = traj[:, 0]
    y_coords = traj[:, 1]
    traj_gt = path[batch_idx]
    traj_gt_x_coords = traj_gt[:, 0]
    traj_gt_y_coords = traj_gt[:, 1]
    
    plt.plot(x_coords, y_coords, 'b-', linewidth=2, alpha=0.7, label='Trajectory')
    plt.plot(traj_gt_x_coords, traj_gt_y_coords, 'r-', linewidth=2, alpha=0.7, label='Trajectory')
    plt.tight_layout()
    # plt.show()
    plt.savefig("model_outputs/plot.jpg")
    

In [21]:
#@markdown ### **Dataset**
#@markdown
#@markdown Defines `PushTImageDataset` and helper functions
#@markdown
#@markdown The dataset class
#@markdown - Load data ((image, agent_pos), action) from a zarr storage
#@markdown - Normalizes each dimension of agent_pos and action to [-1,1]
#@markdown - Returns
#@markdown  - All possible segments with length `pred_horizon`
#@markdown  - Pads the beginning and the end of each episode with repetition
#@markdown  - key `image`: shape (obs_hoirzon, 3, 96, 96)
#@markdown  - key `agent_pos`: shape (obs_hoirzon, 2)
#@markdown  - key `action`: shape (pred_horizon, 2)

def create_sample_indices(
        episode_ends:np.ndarray, sequence_length:int,
        pad_before: int=0, pad_after: int=0):
    indices = list()
    for i in range(len(episode_ends)):
        start_idx = 0
        if i > 0:
            start_idx = episode_ends[i-1]
        end_idx = episode_ends[i]
        episode_length = end_idx - start_idx

        min_start = -pad_before
        max_start = episode_length - sequence_length + pad_after

        # range stops one idx before end
        for idx in range(min_start, max_start+1):
            buffer_start_idx = max(idx, 0) + start_idx
            buffer_end_idx = min(idx+sequence_length, episode_length) + start_idx
            start_offset = buffer_start_idx - (idx+start_idx)
            end_offset = (idx+sequence_length+start_idx) - buffer_end_idx
            sample_start_idx = 0 + start_offset
            sample_end_idx = sequence_length - end_offset
            indices.append([
                buffer_start_idx, buffer_end_idx,
                sample_start_idx, sample_end_idx])
    indices = np.array(indices)
    return indices


def sample_sequence(train_data, sequence_length,
                    buffer_start_idx, buffer_end_idx,
                    sample_start_idx, sample_end_idx):
    result = dict()
    for key, input_arr in train_data.items():
        sample = input_arr[buffer_start_idx:buffer_end_idx]
        data = sample
        if (sample_start_idx > 0) or (sample_end_idx < sequence_length):
            data = np.zeros(
                shape=(sequence_length,) + input_arr.shape[1:],
                dtype=input_arr.dtype)
            if sample_start_idx > 0:
                data[:sample_start_idx] = sample[0]
            if sample_end_idx < sequence_length:
                data[sample_end_idx:] = sample[-1]
            data[sample_start_idx:sample_end_idx] = sample
        result[key] = data
    return result

# normalize data
def get_data_stats(data):
    data = data.reshape(-1,data.shape[-1])
    stats = {
        'min': np.min(data, axis=0),
        'max': np.max(data, axis=0)
    }
    return stats

def normalize_data(data, stats):
    # nomalize to [0,1]
    ndata = (data - stats['min']) / (stats['max'] - stats['min'])
    # normalize to [-1, 1]
    ndata = ndata * 2 - 1
    return ndata

def normalize_data_torch(data, stats):
    # nomalize to [0,1]
    ndata = (data - torch.Tensor(stats['min']).to("cuda")) / torch.Tensor(stats['max'] - stats['min']).to("cuda")
    # normalize to [-1, 1]
    ndata = ndata * 2 - 1
    return ndata

def scale_agent_position_in_pixel(data):
    ndata = (data/512)*96 
    return ndata
# def scale_agent_position_in_pixel(data):
#     ndata = (data/512)*96 
#     return ndata
def unnormalize_data(ndata, stats):
    ndata = (ndata + 1) / 2
    data = ndata * (stats['max'] - stats['min']) + stats['min']
    return data

# dataset
class PushTImageDataset(torch.utils.data.Dataset):
    def __init__(self,
                 dataset_path: str,
                 pred_horizon: int,
                 obs_horizon: int,
                 action_horizon: int):

        # read from zarr dataset
        dataset_root = zarr.open(dataset_path, 'r')

        # float32, [0,1], (N,96,96,3)
        train_image_data = dataset_root['data']['img'][:]
        train_image_data = np.moveaxis(train_image_data, -1,1)
        # (N,3,96,96)

        # (N, D)
        train_data = {
            # first two dims of state vector are agent (i.e. gripper) locations
            'agent_pos': scale_agent_position_in_pixel(dataset_root['data']['state'][:,:2]),
            'action': dataset_root['data']['action'][:]
        }
        episode_ends = dataset_root['meta']['episode_ends'][:]

        # compute start and end of each state-action sequence
        # also handles padding
        indices = create_sample_indices(
            episode_ends=episode_ends,
            sequence_length=pred_horizon,
            pad_before=obs_horizon-1,
            pad_after=action_horizon-1)

        # compute statistics and normalized data to [-1,1]
        stats = dict()
        normalized_train_data = dict()
        for key, data in train_data.items():
            stats[key] = get_data_stats(data)
            # if key !="agent_pos":
            normalized_train_data[key] = normalize_data(data, stats[key])
            # else:


        # images are already normalized
        normalized_train_data['image'] = train_image_data

        self.indices = indices
        self.stats = stats
        self.normalized_train_data = normalized_train_data
        self.pred_horizon = pred_horizon
        self.action_horizon = action_horizon
        self.obs_horizon = obs_horizon

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        # get the start/end indices for this datapoint
        buffer_start_idx, buffer_end_idx, \
            sample_start_idx, sample_end_idx = self.indices[idx]

        # get nomralized data using these indices
        nsample = sample_sequence(
            train_data=self.normalized_train_data,
            sequence_length=self.pred_horizon,
            buffer_start_idx=buffer_start_idx,
            buffer_end_idx=buffer_end_idx,
            sample_start_idx=sample_start_idx,
            sample_end_idx=sample_end_idx
        )

        # discard unused observations
        nsample['image'] = nsample['image'][:self.obs_horizon,:]
        nsample['agent_pos'] = nsample['agent_pos']
        # nsample["action"]=
        return nsample


In [22]:
#@markdown ### **Dataset Demo**

# download demonstration data from Google Drive
dataset_path = "/home/kojogyaase/Projects/Research/neural_potential_fields/pusht_cchi_v7_replay.zarr.zip"
# if not os.path.isfile(dataset_path):
#     id = "1KY1InLurpMvJDRb14L9NlXT_fEsCvVUq&confirm=t"
#     gdown.download(id=id, output=dataset_path, quiet=False)

# parameters
pred_horizon = 8
obs_horizon = 1
action_horizon = 8
device="cuda"
#|o|                             observations: 1
#| |a|a|a|a|a|a|a|a|               actions executed: 8
#|p|p|p|p|p|p|p|p|p|p|p|p|p|p|p|p| actions predicted: 16

# create dataset from file
dataset = PushTImageDataset(
    dataset_path=dataset_path,
    pred_horizon=pred_horizon,
    obs_horizon=obs_horizon,
    action_horizon=action_horizon
)
# save training data statistics (min, max) for each dim
stats = dataset.stats

# create dataloader
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=32,
    num_workers=4,
    shuffle=True,
    # accelerate cpu-gpu transfer
    pin_memory=True,
    # don't kill worker process afte each epoch
    persistent_workers=True
)



In [23]:
# visualize data in batch
batch = next(iter(dataloader))
print("batch['image'].shape:", batch['image'].shape)
print("batch['agent_pos'].shape:", batch['agent_pos'].shape)
# print("batch['action'].shape", batch['action'].shape)
# plot_agent_positions(batch["agent_pos"])

batch['image'].shape: torch.Size([32, 1, 3, 96, 96])
batch['agent_pos'].shape: torch.Size([32, 8, 2])


In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RGBTo2DAutoencoder(nn.Module):
    """
    Autoencoder that takes an RGB image (3x96x96) and encodes it to a latent vector,
    then decodes it into a 2D potential field (1x96x96), which is then squeezed to (96x96).
    This version uses GroupNorm for better batch size flexibility.
    """
    def __init__(self, input_channels=3, latent_dim=512):
        super(RGBTo2DAutoencoder, self).__init__()

        # Encoder: RGB Image -> Latent Vector (using GroupNorm instead of BatchNorm)
        # Input: 3 x 96 x 96
        self.encoder = nn.Sequential(
            # Conv1: 96x96 -> 48x48
            nn.Conv2d(input_channels, 64, kernel_size=4, stride=2, padding=1),  # (96 - 4 + 2*1)/2 + 1 = 48
            nn.GroupNorm(8, 64),
            nn.ReLU(True),

            # Conv2: 48x48 -> 24x24
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), # (48 - 4 + 2*1)/2 + 1 = 24
            nn.GroupNorm(16, 128),
            nn.ReLU(True),

            # Conv3: 24x24 -> 12x12
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), # (24 - 4 + 2*1)/2 + 1 = 12
            nn.GroupNorm(32, 256),
            nn.ReLU(True),

            # Conv4: 12x12 -> 6x6
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1), # (12 - 4 + 2*1)/2 + 1 = 6
            nn.GroupNorm(32, 512), # Added GroupNorm here as well
            nn.ReLU(True),

            # Conv5: 6x6 -> 1x1 (final feature map before flatten)
            nn.Conv2d(512, 512, kernel_size=6, stride=1, padding=0), # (6 - 6 + 2*0)/1 + 1 = 1
            nn.ReLU(True),

            nn.Flatten(),  # 512 * 1 * 1 = 512
            nn.Linear(512, latent_dim),
            nn.ReLU(True)
        )

        # Decoder: Latent Vector -> 96x96 2D Field
        self.decoder = nn.Sequential(
            # Project latent vector to a feature map that can be upsampled
            # Start with a 1x1 latent space from the encoder, which expands to 512 channels
            # We need to project it back to a feature map that can be upsampled to 96x96
            # The last encoder conv outputs 512x1x1, so we start the decoder from that point.
            # We need to reverse the operations.
            # Decoder starts from latent_dim -> 512 (same as the last encoder conv layer output channels)
            nn.Linear(latent_dim, 512 * 1 * 1), # Project latent_dim to the starting feature map size
            nn.ReLU(True),
            nn.Unflatten(1, (512, 1, 1)),  # Reshape to (Batch, 512, 1, 1)

            # Up1: 1x1 -> 6x6 (reverse of Conv5: 6x6 -> 1x1)
            nn.ConvTranspose2d(512, 256, kernel_size=6, stride=1, padding=0), # (1-1)*1 - 2*0 + 6 + 0 + 1 = 6
            nn.GroupNorm(32, 256),
            nn.ReLU(True),

            # Up2: 6x6 -> 12x12 (reverse of Conv4: 12x12 -> 6x6)
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1), # (6-1)*2 - 2*1 + 4 + 0 + 1 = 12
            nn.GroupNorm(16, 128),
            nn.ReLU(True),

            # Up3: 12x12 -> 24x24 (reverse of Conv3: 24x24 -> 12x12)
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1), # (12-1)*2 - 2*1 + 4 + 0 + 1 = 24
            nn.GroupNorm(8, 64),
            nn.ReLU(True),

            # Up4: 24x24 -> 48x48 (reverse of Conv2: 48x48 -> 24x24)
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1), # (24-1)*2 - 2*1 + 4 + 0 + 1 = 48
            nn.GroupNorm(4, 32), # Adjusted group norm if necessary
            nn.ReLU(True),

            # Up5: 48x48 -> 96x96 (reverse of Conv1: 96x96 -> 48x48)
            nn.ConvTranspose2d(32, 1, kernel_size=4, stride=2, padding=1), # (48-1)*2 - 2*1 + 4 + 0 + 1 = 96
            nn.Tanh() # Using Tanh to keep the output values typically in a range like -1 to 1
        )

    def forward(self, x):
        latent = self.encoder(x)
        output_2d = self.decoder(latent)
        # Squeeze the channel dimension if it's 1, resulting in (B, 96, 96)
        output_2d = output_2d.squeeze(1)
        return output_2d

In [ ]:
# import torch
# import torch.nn.functional as F
# import matplotlib.pyplot as plt
# import numpy as np
# import os

# def soft_clamp(x, min_val, max_val, temperature=1.0):
#     """
#     Differentiable soft clamp function using sigmoid.
#     """
#     # Normalize to [0, 1]
#     x_norm = (x - min_val) / (max_val - min_val)
#     # Apply sigmoid for soft clamping
#     x_clamped = torch.sigmoid(temperature * (x_norm - 0.5)) * 0.98 + 0.01
#     # Scale back to original range
#     return x_clamped * (max_val - min_val) + min_val

# def differentiable_grid_sample(field, coords, x_min=-5, x_max=5, y_min=-5, y_max=5):
#     """
#     Differentiable bilinear interpolation using grid_sample.
    
#     Args:
#         field: tensor of shape (B, 1, H, W) - batch of potential fields
#         coords: tensor of shape (B, N, 2) - batch of coordinates in world space
        
#     Returns:
#         values: tensor of shape (B, N) - interpolated values
#     """
#     B, _, H, W = field.shape
#     N = coords.shape[1]
    
#     # Normalize coordinates to [-1, 1] for grid_sample
#     x_norm = 2 * (coords[:, :, 0] - x_min) / (x_max - x_min) - 1
#     y_norm = 2 * (coords[:, :, 1] - y_min) / (y_max - y_min) - 1
    
#     # Stack and reshape for grid_sample: (B, N, 1, 2)
#     grid = torch.stack([x_norm, y_norm], dim=-1).unsqueeze(2)
    
#     # Sample from field: (B, 1, N, 1)
#     sampled = F.grid_sample(field, grid, mode='bilinear', padding_mode='border', align_corners=True)
    
#     # Reshape to (B, N)
#     return sampled.squeeze(1).squeeze(-1)

# def compute_gradient_differentiable(fields, coords, x_min=-5, x_max=5, y_min=-5, y_max=5, h=0.01):
#     """
#     Compute gradients differentiably using finite differences with grid_sample.
    
#     Args:
#         fields: tensor of shape (B, 1, H, W) - batch of potential fields
#         coords: tensor of shape (B, 2) - batch of (x, y) coordinates
        
#     Returns:
#         gradients: tensor of shape (B, 2) - batch of gradients
#     """
#     B = fields.shape[0]
#     device = fields.device
    
#     # Create offset coordinates for finite differences
#     coords_expanded = coords.unsqueeze(1)  # (B, 1, 2)
    
#     # Create perturbation vectors
#     dx_vec = torch.tensor([h, 0.0], device=device).expand(B, 1, 2)
#     dy_vec = torch.tensor([0.0, h], device=device).expand(B, 1, 2)
    
#     coords_x_plus = coords_expanded + dx_vec
#     coords_x_minus = coords_expanded - dx_vec
#     coords_y_plus = coords_expanded + dy_vec
#     coords_y_minus = coords_expanded - dy_vec
    
#     # Soft clamp coordinates to stay within bounds
#     coords_x_plus[:, :, 0] = soft_clamp(coords_x_plus[:, :, 0], x_min, x_max)
#     coords_x_minus[:, :, 0] = soft_clamp(coords_x_minus[:, :, 0], x_min, x_max)
#     coords_y_plus[:, :, 1] = soft_clamp(coords_y_plus[:, :, 1], y_min, y_max)
#     coords_y_minus[:, :, 1] = soft_clamp(coords_y_minus[:, :, 1], y_min, y_max)
    
#     # Sample values at perturbed coordinates
#     val_x_plus = differentiable_grid_sample(fields, coords_x_plus, x_min, x_max, y_min, y_max)
#     val_x_minus = differentiable_grid_sample(fields, coords_x_minus, x_min, x_max, y_min, y_max)
#     val_y_plus = differentiable_grid_sample(fields, coords_y_plus, x_min, x_max, y_min, y_max)
#     val_y_minus = differentiable_grid_sample(fields, coords_y_minus, x_min, x_max, y_min, y_max)
    
#     # Compute gradients
#     grad_x = (val_x_plus - val_x_minus) / (2 * h)
#     grad_y = (val_y_plus - val_y_minus) / (2 * h)
    
#     return torch.stack([grad_x.squeeze(), grad_y.squeeze()], dim=-1)

# def find_extrema_differentiable(fields, x_min=-5, x_max=5, y_min=-5, y_max=5, temperature=40):
#     """
#     Find extrema using differentiable operations with high-temperature softmax.
#     This provides a much better approximation to true argmax/argmin.
    
#     Args:
#         fields: tensor of shape (B, H, W) or (B, 1, H, W)
#         temperature: Higher values make the softmax sharper (closer to true argmax)
        
#     Returns:
#         max_coords: tensor of shape (B, 2) - approximate world coordinates of maxima
#         min_coords: tensor of shape (B, 2) - approximate world coordinates of minima
#     """
#     if len(fields.shape) == 3:
#         fields = fields.unsqueeze(1)  # Add channel dimension
    
#     B, _, H, W = fields.shape
#     device = fields.device
    
#     # Create coordinate grids
#     x_coords = torch.linspace(x_min, x_max, W, device=device)
#     y_coords = torch.linspace(y_min, y_max, H, device=device)
#     Y_grid, X_grid = torch.meshgrid(y_coords, x_coords, indexing='ij')
    
#     # Flatten coordinates
#     X_flat = X_grid.flatten()  # (H*W,)
#     Y_flat = Y_grid.flatten()  # (H*W,)
    
#     max_coords = torch.zeros(B, 2, device=device)
#     min_coords = torch.zeros(B, 2, device=device)
    
#     for i in range(B):
#         field = fields[i, 0]  # (H, W)
#         field_flat = field.flatten()  # (H*W,)
        
#         # Normalize field values to prevent overflow with high temperature
#         field_normalized = (field_flat - field_flat.mean()) / (field_flat.std() + 1e-8)
        
#         # For maximum: use high-temperature softmax
#         max_weights = F.softmax(field_normalized * temperature, dim=0)
#         max_x = torch.sum(max_weights * X_flat)
#         max_y = torch.sum(max_weights * Y_flat)
#         max_coords[i] = torch.stack([max_x, max_y])
        
#         # For minimum: use high-temperature softmax on negative normalized values
#         min_weights = F.softmax(-field_normalized * temperature, dim=0)
#         min_x = torch.sum(min_weights * X_flat)
#         min_y = torch.sum(min_weights * Y_flat)
#         min_coords[i] = torch.stack([min_x, min_y])
    
#     return max_coords, min_coords

# def differentiable_gradient_descent_batch(fields, learning_rate=0.01, num_steps=100, 
#                                         num_path_points=10, x_min=-5, x_max=5, 
#                                         y_min=-5, y_max=5, target_end_points=None):
#     """
#     Fully differentiable gradient descent path finding.
    
#     Args:
#         fields: tensor of shape (B, H, W) or (B, 1, H, W) - batch of potential fields
#         target_end_points: Optional tensor of shape (B, 2) - if provided, use these as targets
        
#     Returns:
#         paths: tensor of shape (B, num_path_points, 2) - batch of paths
#     """
#     # Handle input shape
#     if len(fields.shape) == 3:  # (B, H, W)
#         fields = fields.unsqueeze(1)  # Convert to (B, 1, H, W)
#     elif len(fields.shape) == 4 and fields.shape[-1] == 1:  # (B, H, W, 1)
#         fields = fields.permute(0, 3, 1, 2)  # Convert to (B, 1, H, W)
    
#     B, _, H, W = fields.shape
#     device = fields.device
    
#     # Find start and end points - these will be differentiable
#     start_coords, end_coords = find_extrema_differentiable(fields, x_min, x_max, y_min, y_max)
    
#     if target_end_points is not None:
#         end_coords = target_end_points
    
#     # Initialize path storage - we'll store every step
#     all_positions = torch.zeros(B, num_steps + 1, 2, device=device)
#     all_positions[:, 0] = start_coords
    
#     current_positions = start_coords.clone()
    
#     # Gradient descent loop - fully differentiable
#     for step in range(num_steps):
#         # Compute gradients at current positions
#         gradients = compute_gradient_differentiable(fields, current_positions, 
#                                                   x_min, x_max, y_min, y_max)
        
#         # Update positions
#         current_positions = current_positions - learning_rate * gradients
        
#         # Soft clamp to bounds
#         current_positions[:, 0] = soft_clamp(current_positions[:, 0], x_min, x_max)
#         current_positions[:, 1] = soft_clamp(current_positions[:, 1], y_min, y_max)
        
#         all_positions[:, step + 1] = current_positions
    
#     # Differentiable path resampling using interpolation
#     # Create indices for resampling
#     step_indices = torch.linspace(0, num_steps, num_path_points, device=device)
    
#     # Use linear interpolation to get path points at desired indices
#     paths = torch.zeros(B, num_path_points, 2, device=device)
    
#     for i in range(num_path_points):
#         idx = step_indices[i]
        
#         # Get integer parts and fractional part for interpolation
#         idx_floor = torch.floor(idx).long()
#         idx_ceil = torch.ceil(idx).long()
#         alpha = idx - idx_floor.float()
        
#         # Clamp indices
#         idx_floor = torch.clamp(idx_floor, 0, num_steps)
#         idx_ceil = torch.clamp(idx_ceil, 0, num_steps)
        
#         # Linear interpolation between adjacent points
#         if idx_floor == idx_ceil:
#             paths[:, i] = all_positions[:, idx_floor]
#         else:
#             paths[:, i] = (1 - alpha) * all_positions[:, idx_floor] + alpha * all_positions[:, idx_ceil]
    
#     return paths

# def save_batch_plots_differentiable(fields, paths, output_dir="model/outputs", batch_idx=0, 
#                                   x_min=-1, x_max=1, y_min=-1, y_max=1, max_plots=4):
#     """
#     Save plots for a subset of the batch (non-differentiable, for visualization only).
#     """
#     os.makedirs(output_dir, exist_ok=True)
    
#     # Handle different input shapes
#     if len(fields.shape) == 4:
#         if fields.shape[1] == 1:  # (B, 1, H, W)
#             fields_plot = fields.squeeze(1)
#         else:  # (B, H, W, 1)
#             fields_plot = fields.squeeze(-1)
#     else:  # (B, H, W)
#         fields_plot = fields
    
#     B = min(fields_plot.shape[0], max_plots)
    
#     for i in range(B):
#         field = fields_plot[i].detach().cpu().numpy()
#         path = paths[i].detach().cpu().numpy()
        
#         plt.figure(figsize=(10, 8))
#         plt.imshow(field, origin='lower', cmap='viridis', 
#                    extent=[x_min, x_max, y_min, y_max])
#         plt.colorbar(label='Potential Value')
#         plt.title(f'Batch {batch_idx}, Sample {i}: Differentiable Path')
#         plt.xlabel('X')
#         plt.ylabel('Y')

#         # Plot path
#         plt.plot(path[:, 0], path[:, 1], 'r--', linewidth=2, label='Differentiable Path')
#         plt.plot(path[:, 0], path[:, 1], 'go', markersize=4, label='Path Points')
#         plt.plot(path[0, 0], path[0, 1], 'ro', markersize=8, label='Start')
#         plt.plot(path[-1, 0], path[-1, 1], 'bx', markersize=8, label='End')
        
#         plt.legend()
#         plt.grid(True)
#         plt.tight_layout()
#         plt.savefig(os.path.join(output_dir, f"differentiable_batch_{batch_idx}_sample_{i}.png"), 
#                    dpi=300, bbox_inches='tight')
#         plt.close()

# def process_batch_fields_differentiable(field_batch, learning_rate=0.05, num_steps=100, 
#                                       num_path_points=10, output_dir="model/outputs", 
#                                       batch_idx=0, save_plots_flag=False, 
#                                       x_min=-1, x_max=1, y_min=-1, y_max=1):
#     """
#     Process a batch of potential fields to find differentiable gradient descent paths.
    
#     Args:
#         field_batch: tensor of shape (B, 96, 96, 1) or (B, 96, 96) or (B, 1, 96, 96)
        
#     Returns:
#         paths: tensor of shape (B, num_path_points, 2) - batch of paths (differentiable)
#     """
#     # Find paths for the entire batch
#     paths = differentiable_gradient_descent_batch(
#         field_batch, 
#         learning_rate=learning_rate, 
#         num_steps=num_steps,
#         num_path_points=num_path_points,
#         x_min=x_min, x_max=x_max, y_min=y_min, y_max=y_max
#     )
    
#     # Save plots if requested (this part is not differentiable)
#     if save_plots_flag:
#         save_batch_plots_differentiable(field_batch, paths, output_dir, batch_idx, 
#                                       x_min, x_max, y_min, y_max)
    
#     return paths



In [ ]:
model=RGBTo2DAutoencoder()
# field = model(torch.randn((1,3,96,96)))
# shape=field.shape
# paths=[]
# for _ in  range(shape[0]):
# path = process_batch_fields_differentiable(
#         field,
#         learning_rate=0.05,
#         num_steps=50,  # Reduced for efficiency
#         num_path_points=10,
#         output_dir="model/outputs",
#         batch_idx=0,
#         save_plots_flag=True
# )

model=model.to(device)


In [ ]:
# import torch

# def soft_clamp(x, min_val, max_val, temperature=1.0):
#     x_norm = (x - min_val) / (max_val - min_val)
#     return (torch.sigmoid(temperature * (x_norm - 0.5)) * 0.98 + 0.01) * (max_val - min_val) + min_val

# def linear_interp1d(x, xp, yp):
#     indices = torch.searchsorted(xp, x).clamp(1, len(xp) - 1)
#     x1, x2 = xp[indices-1], xp[indices]
#     y1, y2 = yp[indices-1], yp[indices]
#     return y1 + (x - x1) * (y2 - y1) / (x2 - x1 + 1e-8)

# def compute_differentiable_path(potential_field, step_size=0.1, num_simulation_steps=100, num_path_points=10, convergence_threshold=1e-4):
#     B, H, W = potential_field.shape
#     device = potential_field.device
    
#     flat_potentials = potential_field.view(B, -1)
#     start_indices = torch.argmax(flat_potentials, dim=1)
#     end_indices = torch.argmin(flat_potentials, dim=1)
    
#     start_pos = torch.stack([(start_indices % W).float(), (start_indices // W).float()], dim=1)
#     end_pos = torch.stack([(end_indices % W).float(), (end_indices // W).float()], dim=1)
    
#     current_pos = start_pos.clone()
#     converged = torch.zeros(B, dtype=torch.bool, device=device)
#     path_lengths = torch.full((B,), num_simulation_steps, device=device)
#     all_positions = torch.zeros(num_simulation_steps+1, B, 2, device=device)
#     all_positions[0] = start_pos

#     for step in range(1, num_simulation_steps+1):
#         gradients = compute_fast_gradient(potential_field, current_pos)
#         step_direction = -gradients / (torch.norm(gradients, dim=1, keepdim=True) + 1e-8)
        
#         new_pos = current_pos + step_size * step_direction
#         new_pos = torch.stack([
#             torch.clamp(new_pos[:, 0], 0.0, W-1.0),
#             torch.clamp(new_pos[:, 1], 0.0, H-1.0)
#         ], dim=1)
        
#         distances = torch.norm(new_pos - end_pos, dim=1)
#         newly_converged = (distances < convergence_threshold) & ~converged
#         path_lengths = torch.where(newly_converged, step, path_lengths)
#         converged = converged | newly_converged
        
#         current_pos = torch.where(converged.unsqueeze(1), current_pos, new_pos)
#         all_positions[step] = current_pos
        
#         if converged.all():
#             all_positions = all_positions[:step+1]
#             break

#     final_paths = torch.zeros(B, num_path_points, 2, device=device)
#     interp_indices = torch.linspace(0, 1, num_path_points, device=device)
    
#     for b in range(B):
#         n_steps = min(path_lengths[b].item()+1, all_positions.shape[0])
#         path = all_positions[:n_steps, b]
#         t = torch.linspace(0, 1, n_steps, device=device)
#         for dim in range(2):
#             final_paths[b, :, dim] = linear_interp1d(interp_indices, t, path[:, dim])
    
#     return final_paths

# def compute_fast_gradient(potential_field, positions):
#     B, H, W = potential_field.shape
#     x, y = positions[:, 0], positions[:, 1]
#     eps = 0.5
    
#     x_plus = torch.clamp(x + eps, 0.0, W-1.0)
#     x_minus = torch.clamp(x - eps, 0.0, W-1.0)
#     y_plus = torch.clamp(y + eps, 0.0, H-1.0)
#     y_minus = torch.clamp(y - eps, 0.0, H-1.0)
    
#     val_x_plus = bilinear_sample_batch(potential_field, x_plus, y)
#     val_x_minus = bilinear_sample_batch(potential_field, x_minus, y)
#     val_y_plus = bilinear_sample_batch(potential_field, x, y_plus)
#     val_y_minus = bilinear_sample_batch(potential_field, x, y_minus)
    
#     return torch.stack([
#         (val_x_plus - val_x_minus) / (x_plus - x_minus + 1e-8),
#         (val_y_plus - val_y_minus) / (y_plus - y_minus + 1e-8)
#     ], dim=1)

# def bilinear_sample_batch(field, x, y):
#     B, H, W = field.shape
#     x0 = torch.floor(x).long().clamp(0, W-2)
#     y0 = torch.floor(y).long().clamp(0, H-2)
#     x1, y1 = x0 + 1, y0 + 1
    
#     wx = x - x0.float()
#     wy = y - y0.float()
    
#     return ((1-wx)*(1-wy)*field[torch.arange(B), y0, x0] + 
#             wx*(1-wy)*field[torch.arange(B), y0, x1] + 
#             (1-wx)*wy*field[torch.arange(B), y1, x0] + 
#             wx*wy*field[torch.arange(B), y1, x1])



In [ ]:


# Training loop
import random
import numpy as np
import torch.nn as nn
from tqdm import tqdm

def train_model(model, dataloader, device, obs_horizon, action_horizon, num_epochs=10):
    """Train the neural network model."""
    optimizer = torch.optim.Adam(
        params=model.parameters(),
        lr=1e-2, 
        # weight_decay=1e-6
    )
    
    epoch_losses = []
    batch_count = 0
    
    with tqdm(range(num_epochs), desc='Epoch') as tglobal:
        for epoch_idx in tglobal:
            epoch_loss = []
            
            with tqdm(dataloader, desc='Batch', leave=False) as tepoch:
                for batch_idx, nbatch in enumerate(tepoch):
                    batch_count += 1
                    
                    # Data transfer to device
                    nimage = nbatch['image'][:, :obs_horizon].to(device).squeeze(1)
                    nagent_pos = nbatch['agent_pos'][:, :obs_horizon].to(device)
                    naction = nbatch['agent_pos'].to(device)
                    
                    # Forward pass
                    field = model(nimage)
                    B = nagent_pos.shape[0]
                    
                    # Process batch fields differentiably
                    path = compute_differentiable_path(
                        field,
                        num_path_points=action_horizon
                    ).to(device)
                    
                    # Normalize path if needed
                    normalized_path = normalize_data_torch(path,dataset.stats["agent_pos"])
                    
                    # Visualization (occasional)
                    if batch_idx % 50 == 0:
                        try:
                            plot_agent_positions(naction, normalized_path)
                            viz(field.clone().detach().cpu()[random.randint(0, min(63, B-1))])
                        except:
                            pass  # Skip visualization if functions not available
                    
                    # Compute loss
                    loss = nn.functional.mse_loss(normalized_path, naction)
                    
                    # Backward pass and optimization
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    
                    # Logging
                    loss_cpu = loss.item()
                    epoch_loss.append(loss_cpu)
                    
                    batch_avg_loss = np.mean(epoch_loss)
                    tepoch.set_postfix({
                        'batch_loss': f'{loss_cpu:.6f}',
                        'batch_avg': f'{batch_avg_loss:.6f}',
                        'batch': f'{batch_idx+1}/{len(dataloader)}'
                    })
                    
                    # # Print detailed batch stats every 20 batches
                    if (batch_idx + 1) % 20 == 0:
                        print(f"\nBatch {batch_idx+1}/{len(dataloader)} | "
                              f"Loss: {loss_cpu:.6f} | "
                              f"Avg Loss: {batch_avg_loss:.6f} | "
                              f"Field Shape: {field.shape} | "
                              f"Path Shape: {path.shape}")
            
            # Epoch statistics
            epoch_avg_loss = np.mean(epoch_loss)
            epoch_losses.append(epoch_avg_loss)
            
            tglobal.set_postfix({
                'epoch_loss': f'{epoch_avg_loss:.6f}',
                'best_loss': f'{min(epoch_losses):.6f}',
                'batches': len(dataloader)
            })
    
    return epoch_losses

# Usage example:
epoch_losses = train_model(model, dataloader, device, obs_horizon, action_horizon, num_epochs=30)

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]


Batch 20/802 | Loss: 0.587764 | Avg Loss: 0.450931 | Field Shape: torch.Size([32, 96, 96]) | Path Shape: torch.Size([32, 8, 2])



Batch 40/802 | Loss: 0.650118 | Avg Loss: 0.530918 | Field Shape: torch.Size([32, 96, 96]) | Path Shape: torch.Size([32, 8, 2])



Batch 60/802 | Loss: 0.699989 | Avg Loss: 0.482667 | Field Shape: torch.Size([32, 96, 96]) | Path Shape: torch.Size([32, 8, 2])



Batch 80/802 | Loss: 0.716899 | Avg Loss: 0.522848 | Field Shape: torch.Size([32, 96, 96]) | Path Shape: torch.Size([32, 8, 2])


...


Batch 120/802 | Loss: 1.405366 | Avg Loss: 1.359037 | Field Shape: torch.Size([32, 96, 96]) | Path Shape: torch.Size([32, 8, 2])



Batch 140/802 | Loss: 1.237107 | Avg Loss: 1.364655 | Field Shape: torch.Size([32, 96, 96]) | Path Shape: torch.Size([32, 8, 2])



Batch 160/802 | Loss: 1.398776 | Avg Loss: 1.368288 | Field Shape: torch.Size([32, 96, 96]) | Path Shape: torch.Size([32, 8, 2])



Batch 180/802 | Loss: 1.386072 | Avg Loss: 1.368524 | Field Shape: torch.Size([32, 96, 96]) | Path Shape: torch.Size([32, 8, 2])



Batch 200/802 | Loss: 1.460121 | Avg Loss: 1.370697 | Field Shape: torch.Size([32, 96, 96]) | Path Shape: torch.Size([32, 8, 2])


In [ ]:
# #@markdown ### **Inference**

# # limit enviornment interaction to 200 steps before termination
# max_steps = 200
# env = PushTImageEnv()
# # use a seed >200 to avoid initial states seen in the training dataset
# Batch 20/802 | Loss: 0.639597 | Avg Loss: 0.438793 | Field Shape: torch.Size([32, 96, 96]) | Path Shape: torch.Size([32, 8, 2])
env.seed(100000)

# # get first observation
# obs, info = env.reset()

# # keep a queue of last 2 steps of observations
# obs_deque = collections.deque(
#     [obs] * obs_horizon, maxlen=obs_horizon)
# # save visualization and rewards
# imgs = [env.render(mode='rgb_array')]
# rewards = list()
# done = False
# step_idx = 0

# with tqdm(total=max_steps, desc="Eval PushTImageEnv") as pbar:
#     while not done:
#         B = 1
#         # stack the last obs_horizon number of observations
#         images = np.stack([x['image'] for x in obs_deque])
#         agent_poses = np.stack([x['agent_pos'] for x in obs_deque])

#         # normalize observation
#         nagent_poses = normalize_data(agent_poses, stats=stats['agent_pos'])
#         # images are already normalized to [0,1]
#         nimages = images

#         # device transfer
#         nimages = torch.from_numpy(nimages).to(device, dtype=torch.float32)
#         # (2,3,96,96)
#         nagent_poses = torch.from_numpy(nagent_poses).to(device, dtype=torch.float32)
#         # (2,2)

#         # infer action
#         with torch.no_grad():
#             # get image features
#             image_features = ema_nets['vision_encoder'](nimages)
#             # (2,512)

#             # concat with low-dim observations
#             obs_features = torch.cat([image_features, nagent_poses], dim=-1)

#             # reshape observation to (B,obs_horizon*obs_dim)
#             obs_cond = obs_features.unsqueeze(0).flatten(start_dim=1)

#             # initialize action from Guassian noise
#             noisy_action = torch.randn(
#                 (B, pred_horizon, action_dim), device=device)
#             naction = noisy_action

#             # init scheduler
#             noise_scheduler.set_timesteps(num_diffusion_iters)

#             for k in noise_scheduler.timesteps:
#                 # predict noise
#                 noise_pred = ema_nets['noise_pred_net'](
#                     sample=naction,
#                     timestep=k,
#                     global_cond=obs_cond
#                 )

#                 # inverse diffusion step (remove noise)
#                 naction = noise_scheduler.step(
#                     model_output=noise_pred,
#                     timestep=k,
#                     sample=naction
#                 ).prev_sample

#         # unnormalize action
#         naction = naction.detach().to('cpu').numpy()
#         # (B, pred_horizon, action_dim)
#         naction = naction[0]
#         action_pred = unnormalize_data(naction, stats=stats['action'])

#         # only take action_horizon number of actions
#         start = obs_horizon - 1
#         end = start + action_horizon
#         action = action_pred[start:end,:]
#         # (action_horizon, action_dim)

#         # execute action_horizon number of steps
#         # without replanning
#         for i in range(len(action)):
#             # stepping env
#             obs, reward, done, _, info = env.step(action[i])
#             # save observations
#             obs_deque.append(obs)
#             # and reward/vis
#             rewards.append(reward)
#             imgs.append(env.render(mode='rgb_array'))

#             # update progress bar
#             step_idx += 1
#             pbar.update(1)
#             pbar.set_postfix(reward=reward)
#             if step_idx > max_steps:
#                 done = True
#             if done:
#                 break

# # print out the maximum target coverage
# print('Score: ', max(rewards))

# # visualize
# from IPython.display import Video
# vwrite('vis.mp4', imgs)
# Video('vis.mp4', embed=True, width=256, height=256)